In [ ]:
import pandas as pd
oddsdf = pd.read_csv(r'C:\Users\jiabi\NBA_Analyzerv2\kaggle_odds_combined.csv', low_memory=False)
playersdf = pd.read_csv(r'C:\Users\jiabi\NBA_Analyzerv2\nba_data.csv', low_memory=False)
oddsdf = oddsdf.drop_duplicates()
playersdf = playersdf.drop_duplicates()

In [ ]:
oddsdf.columns.tolist()

In [ ]:
playersdf.columns.tolist()

In [ ]:
columnsToFetch = [
    'game_date',
    'NBA_team_name',
    'home_away',
    'close_moneyline_line',
    'close_spread_line',
    'close_total_line'
]
#Filter by columns of data I am going to use
oddsdf = oddsdf[columnsToFetch]

oddsdf = oddsdf.dropna(subset=['close_spread_line', 'close_moneyline_line'])

oddsdf = oddsdf.reset_index(drop=True)

print(f"Total Rows: {len(oddsdf)}")
oddsdf.head()

In [ ]:
#Filtering by dates since it had extraneous dates beyond the description of the kaggle set
oddsdf['game_date'] = pd.to_datetime(oddsdf['game_date'])
start_date = '2021-12-2'
end_date = '2022-12-11'
oddsdf['game_date'] = pd.to_datetime(oddsdf['game_date'])
oddsdf = oddsdf.loc[(oddsdf['game_date'] >= start_date) & (oddsdf['game_date'] < end_date)]
oddsdf = oddsdf.sort_values(by='game_date')
oddsdf = oddsdf.reset_index(drop=True)
print(f"Total Rows: {len(oddsdf)}")
oddsdf.head(10)

In [ ]:
# From the blocks below, found that Los Angeles Clippers was named to something else, so I had to change naming before merging tables
print(playersdf[playersdf['TEAM_NAME'].str.contains("Clippers", na=False)]['TEAM_NAME'].unique())

In [ ]:
columnsFetch= [
    'TEAM_NAME',
    'GAME_DATE',
    'MATCHUP',
    'WL',
    'PLUS_MINUS',
    'FG_PCT',
    'FG3_PCT',
    'FT_PCT',
    'REB',
    'OREB',
    'DREB',
    'AST',
    'TOV',
    'STL',
    'BLK',
    'PF',
    'PTS'
]
#Columns of data I want to analyze for NBA
nba_teams = [
    "Atlanta Hawks",
    "Boston Celtics",
    "Brooklyn Nets",
    "Charlotte Hornets",
    "Chicago Bulls",
    "Cleveland Cavaliers",
    "Dallas Mavericks",
    "Denver Nuggets",
    "Detroit Pistons",
    "Golden State Warriors",
    "Houston Rockets",
    "Indiana Pacers",
    "LA Clippers",
    "Los Angeles Lakers",
    "Memphis Grizzlies",
    "Miami Heat",
    "Milwaukee Bucks",
    "Minnesota Timberwolves",
    "New Orleans Pelicans",
    "New York Knicks",
    "Oklahoma City Thunder",
    "Orlando Magic",
    "Philadelphia 76ers",
    "Phoenix Suns",
    "Portland Trail Blazers",
    "Sacramento Kings",
    "San Antonio Spurs",
    "Toronto Raptors",
    "Utah Jazz",
    "Washington Wizards"
]
#The original data contained teams from other leagues, e.g., the G-League NBA teams, so I had to filter for NBA Teams
team_df = playersdf[columnsFetch].dropna().reset_index(drop=True)
team_df = team_df[team_df['TEAM_NAME'].isin(nba_teams)]
print(f"Total Rows:{len(team_df)}")
team_df.head(10)

In [ ]:
#Filtering by dates to match the date range of the betting lines data
start_date = '2021-12-2'
end_date = '2022-12-11'
team_df['GAME_DATE'] = pd.to_datetime(team_df['GAME_DATE'])
team_df = team_df.loc[(team_df['GAME_DATE'] >= start_date) & (team_df['GAME_DATE'] < end_date)]
team_df = team_df.reset_index(drop=True)
print(f"Total Rows: {len(team_df)}")
team_df.head(10)

In [ ]:
#Checking if the names match before merging the tables
print("Odds Teams:", sorted(oddsdf['NBA_team_name'].unique()))
print("Player Teams:", sorted(team_df['TEAM_NAME'].unique()))

In [ ]:
#Merging odds and team dataframes
merged_df = pd.merge(team_df, oddsdf, left_on=['TEAM_NAME', 'GAME_DATE'], right_on=['NBA_team_name', 'game_date'], how='left')
print(f"Rows in original team_df: {len(team_df)}")
print(f"Rows in merged_df: {len(merged_df)}")
#Dropped merged columns because they are duplicates
merged_df = merged_df.drop(['game_date','NBA_team_name'], axis=1)
merged_df.head(10)